In [25]:

import numpy as np
from os import listdir
from skimage import io
from scipy.misc import imresize
from keras.preprocessing.image import array_to_img, img_to_array, load_img

def get_img(data_path):
    # Getting image array from path:
    img_size = 64
    img = io.imread(data_path)
    img = imresize(img, (img_size, img_size, 3))
    return img

def get_dataset(dataset_path=r'F:\js\Dog-Cat-Classifier\Data\Train_Data'):
    # Getting all data from data path:
    try:
        X = np.load(r'F:\js\Dog-Cat-Classifier\Data\npy_train_data\X.npy')
        Y = np.load(r'F:\js\Dog-Cat-Classifier\Data\npy_train_data\Y.npy')
    except:
        labels = listdir(dataset_path) # Geting labels
        print('Categories:\n', labels)
        len_datas = 0
        for label in labels:
            len_datas += len(listdir(dataset_path+'/'+label))
        X = np.zeros((len_datas, 64, 64, 3), dtype='float64')
        Y = np.zeros(len_datas)
        count_data = 0
        count_categori = [-1,''] # For encode labels
        for label in labels:
            datas_path = dataset_path+'/'+label
            for data in listdir(datas_path):
                img = get_img(datas_path+'/'+data)
                X[count_data] = img
                # For encode labels:
                if label != count_categori[1]:
                    count_categori[0] += 1
                    count_categori[1] = label
                Y[count_data] = count_categori[0]
                count_data += 1
        # Create dateset:
        import keras
        Y = keras.utils.to_categorical(Y)
        import os
        if not os.path.exists('F:\js\Dog-Cat-Classifier\Data/npy_train_data/'):
            os.makedirs('F:\js\Dog-Cat-Classifier\Data/npy_train_data/')
        np.save(r'F:\js\Dog-Cat-Classifier\Data\npy_train_data\X.npy', X)
        np.save(r'F:\js\Dog-Cat-Classifier\Data\npy_train_data\Y.npy', Y)
        
    from sklearn.model_selection import train_test_split
    X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
    return X, X_test, Y, Y_test


In [27]:

import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

def save_model(model):
    if not os.path.exists('F:\js\Dog-Cat-Classifier\Data/Model/'):
        os.makedirs('F:\js\Dog-Cat-Classifier\Data/Model/')
    model_json = model.to_json()
    with open("F:\js\Dog-Cat-Classifier\Data/Model/model.json", "w") as model_file:
        model_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("Data/Model/weights.h5")
    print('Model and weights saved')
    return

def get_model(num_classes=2):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return model

save_model(get_model())


In [28]:

import os
from keras.callbacks import ModelCheckpoint, TensorBoard

def train_model(model, X, X_test, Y, Y_test):
    checkpoints = []
    if not os.path.exists('F:\js\Dog-Cat-Classifier\Data/Checkpoints/'):
        os.makedirs('F:\js\Dog-Cat-Classifier\Data/Checkpoints/')
    checkpoints.append(ModelCheckpoint('F:\js\Dog-Cat-Classifier\Data/Checkpoints/best_weights.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1))
    checkpoints.append(TensorBoard(log_dir='F:\js\Dog-Cat-Classifier\Data/Checkpoints/./logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None))

    # Creates live data:
    # For better yield. The duration of the training is extended.

    # If you don't want, use this:
    # model.fit(X, Y, batch_size=10, epochs=25, validation_data=(X_test, Y_test), shuffle=True, callbacks=checkpoints)

    from keras.preprocessing.image import ImageDataGenerator
    generated_data = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, zca_whitening=False, rotation_range=0,  width_shift_range=0.1, height_shift_range=0.1, horizontal_flip = True, vertical_flip = False)
    generated_data.fit(X)
    import numpy
    model.fit_generator(generated_data.flow(X, Y, batch_size=10), steps_per_epoch=X.shape[0], epochs=25, validation_data=(X_test, Y_test), callbacks=checkpoints)

    return model



In [29]:
X, X_test, Y, Y_test = get_dataset()
model = get_model(len(Y[0]))
import numpy
model = train_model(model, X, X_test, Y, Y_test)
save_model(model)



Epoch 1/25
1260/1260 [==============================] - 109s 86ms/step - loss: 1.3534 - acc: 0.5226 - val_loss: 0.6886 - val_acc: 0.4929
Epoch 2/25
1260/1260 [==============================] - 84s 67ms/step - loss: 0.6973 - acc: 0.5352 - val_loss: 0.6731 - val_acc: 0.5250
Epoch 3/25
1260/1260 [==============================] - 84s 67ms/step - loss: 0.6955 - acc: 0.5433 - val_loss: 0.6856 - val_acc: 0.4929
Epoch 4/25
1260/1260 [==============================] - 85s 68ms/step - loss: 0.6908 - acc: 0.5670 - val_loss: 0.6894 - val_acc: 0.5000
Epoch 5/25
1260/1260 [==============================] - 85s 67ms/step - loss: 0.6854 - acc: 0.5820 - val_loss: 0.6587 - val_acc: 0.5679
Epoch 6/25
1260/1260 [==============================] - 84s 67ms/step - loss: 0.6369 - acc: 0.6466 - val_loss: 0.6464 - val_acc: 0.5679
Epoch 7/25
1260/1260 [==============================] - 84s 67ms/step - loss: 0.6134 - acc: 0.6700 - val_loss: 0.6268 - val_acc: 0.6107
Epoch 8/25
1260/1260 [=========================

In [30]:

from keras.models import Sequential
from keras.models import model_from_json

def predict(model, X):
    return model.predict(X)



In [31]:
import sys
img_dir = sys.argv[1]
img = get_img(r'F:\js\Dog-Cat-Classifier\test_dog.jpg')

import numpy as np
X = np.zeros((1, 64, 64, 3), dtype='float64')
X[0] = img
# Getting model:
model_file = open('F:\js\Dog-Cat-Classifier\Data\Model\model.json', 'r')
model = model_file.read()
model_file.close()
model = model_from_json(model)
# Getting weights
model.load_weights("F:\js\Dog-Cat-Classifier\Data\Model\weights.h5")
print('Possibilities:\n[[ <Cat>  <Dog> ]]\n' + str(predict(model, X)))


c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]


Possibilities:
[[ <Cat>  <Dog> ]]
[[0.04618423 0.9203562 ]]
